In [1]:
import os

In [2]:
%pwd

'd:\\Text-Summarizer-Project\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\Text-Summarizer-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)

class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
import sys
sys.path.append("D:\Text-Summarizer-Project\src\TextSummarizer")


In [7]:
!pip install TextSummarizer

In [9]:
from src.TextSummarizer.constants import *

In [13]:
%pip install boxsdk

Note: you may need to restart the kernel to use updated packages.


In [26]:
%pip install --upgrade python-box 
%pip install PyYAML
%pip install ensure

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [27]:
from src.TextSummarizer.utils.common import read_yaml, create_dir

In [28]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        
        create_dir([self.config.artifacts_root])

        